In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

**Create folders and unzip**

In [ ]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm

main_folder = "/content/drive/My Drive/IBD/"
unzipped_data_folder =  os.path.join(main_folder, "data_03112020_unzipped")
original_files_folder = os.path.join(main_folder, "data_all")

os.mkdir(unzipped_data_folder)
os.mkdir(original_files_folder)

In [ ]:
! unzip /content/drive/My\ Drive/IBD/data_03112020/*  -d /content/drive/My\ Drive/IBD/data_03112020_unzipped

**Process single folder**

In [ ]:
sample_video='UC10-0.mp4'

filename=os.path.join(unzipped_data_folder, sample_video, 'objects.json') 
with open(filename) as f:
    object_anns = json.load(f)

filename=os.path.join(unzipped_data_folder, sample_video, 'classifications.json')
with open(filename) as f:
    class_anns = json.load(f)

obj_frame_labels=object_anns['frame_labels']
class_frame_labels=class_anns['frame_labels']

video_link=object_anns['data_link']

os.mkdir(os.path.join(original_files_folder, sample_video))
os.mkdir(os.path.join(original_files_folder, sample_video, "frames"))
os.mkdir(os.path.join(original_files_folder, sample_video, "video"))

cap = cv2.VideoCapture(video_link)

frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
fps = int(cap.get(5))

video_file = cv2.VideoWriter(os.path.join(original_files_folder, sample_video, "video", "video.mp4"),cv2.VideoWriter_fourcc(*'MJPG'), fps, size)

frame_count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if(not ret):
        break    
         
    video_file.write(frame)   
    frame = cv2.resize(frame, None, fx=0.25, fy=0.25, interpolation = cv2.INTER_AREA)
    cv2.imwrite(os.path.join(original_files_folder, sample_video, "frames", str(frame_count)+".bmp"), frame)
    frame_count += 1

cap.release()
video_file.release()

**Process all folders**

In [ ]:
folder_names = os.listdir(unzipped_data_folder)

for folder in tqdm(folder_names):
    filename=os.path.join(unzipped_data_folder, folder, 'objects.json') 
    with open(filename) as f:
        object_anns = json.load(f)
    
    video_link=object_anns['data_link']

    os.mkdir(os.path.join(original_files_folder, folder))
    os.mkdir(os.path.join(original_files_folder, folder, "frames"))
    os.mkdir(os.path.join(original_files_folder, folder, "video"))

    cap = cv2.VideoCapture(video_link)

    frame_width = int(cap.get(3)) 
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)
    fps = int(cap.get(5))

    video_file = cv2.VideoWriter(os.path.join(original_files_folder, folder, "video", "video.mp4"),cv2.VideoWriter_fourcc(*'MJPG'), fps, size)

    frame_count = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if(not ret):
            break     
            
        video_file.write(frame)   
        frame = cv2.resize(frame, None, fx=0.25, fy=0.25, interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(original_files_folder, folder, "frames", str(frame_count)+".bmp"), frame)
        frame_count += 1

    cap.release()
    video_file.release()


**Remove leaked files**


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
import shutil
import os
for item in os.scandir("/content/drive/My Drive"):
    if item.is_file():
        if item.name.endswith(".bmp") or item.name.endswith(".mp4"):
            print(item.path)
            os.remove(item.path)
    elif item.is_dir():
        if item.name.startswith("frame") or item.name.startswith("video"):
            print(item.path)
            shutil.rmtree(item.path)      